In [10]:
import requests

baseURL = 'https://nguyenduchuy.pythonanywhere.com'
def get_products(category):
    url = f"{baseURL}/products/get-all-products".format(baseURL)  
    params = {"category": category}
    response = requests.get(url, params=params)

    return response.text
result = get_products("best_seller")
result

'<!doctype html>\n<html lang=en>\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'

In [ ]:
import requests

baseURL = 'https://nguyenduchuy.pythonanywhere.com'
def get_products(category):
    url = f"{baseURL}/products/get-all-products".format(baseURL)  
    params = {"category": category}
    response = requests.get(url, params=params)

    return response.text
result = get_products({})
result

'hello world'

In [5]:
import requests

def search_products(category):
    url = "http://127.0.0.1:5000/products/search-product"  # Replace with your actual URL
    params = {"searchTerm": category}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        # Handle errors based on response status code
        print(f"Error: {response.status_code}")
        return None
result = search_products("phone")
result

{'result': [{'categoryDetails': {'_key': 'best_seller',
    'categoryName': 'Best Sellers',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'},
   'productDetails': {'_key': 'prod_456',
    'productCategoryId': ['best_seller', 'free_delivery'],
    'productDescription': 'A high-quality smartphone with advanced features.',
    'productName': 'Smartphone Z',
    'productPicture': 'http://127.0.0.1:5000/static/images/iphone11promax.png',
    'reviews': [{'comment': 'Great phone!', 'userId': 'user_789'}],
    'variations': [{'availabilityQuantity': 10,
      'discountPrice': 0.0,
      'offerPrice': 999.0,
      'quantity': 1,
      'sellingPrice': 1099.0}]}},
  {'categoryDetails': {'_key': 'free_delivery',
    'categoryName': 'Free Delivery',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'},
   'productDetails': {'_key': 'prod_456',
    'productCategoryId': ['best_seller', 'free_delivery'],
    'productDescription': 'A high-quality smartphone wi

In [ ]:
{'result': [{
    'categoryDetails': {
        '_key': 'free_delivery',
        'categoryName': 'Free Delivery',
        'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'
    },
        'productDetails': {
            '_key': 'prod_456',
            'productDescription': 'A high-quality smartphone with advanced features.',
            'productName': 'Smartphone Z',
            'productPicture': 'http://127.0.0.1:5000/static/images/iphone11promax.png',
            'reviews': [
                    {
                    'comment': 'Great phone!', 'userId': 'user_789'
                    }
                ],
            'variations': [
                    {'availabilityQuantity': 10,
                    'discountPrice': 0.0,
                    'offerPrice': 999.0,
                    'quantity': 1,
                    'sellingPrice': 1099.0
                    }
                ]
            }
        }
    ]
 }

In [19]:
import sqlite3
from flask import jsonify
import threading
import json

# Thread-local storage for database connections
thread_local = threading.local()

def get_db_connection(path='backend/assets/database/database.db'):
    if not hasattr(thread_local, 'db_conn'):
        thread_local.db_conn = sqlite3.connect(path)
    return thread_local.db_conn
   

def get_product_from_key(key):  
    '''
    Function to get all information of products by arbitrary key.
    '''
    # Connect to DB
    connection = get_db_connection()
    cursor = connection.cursor()

    # Set key to search
    key_name = 'p' # Default querying by product key
    if key['type'] == 'category':
        key_name = 'c'
        
    # Query to get category details (use JOIN for efficiency)
    query = """
        SELECT p._key, p.productName, p.productDescription, p.productPicture
        FROM categories c
        INNER JOIN product_categories pc ON c._key = pc.category_key
        INNER JOIN products p ON pc.product_key = p._key
        WHERE {key_name}._key = ? """.format(key_name=key_name)
    
    cursor.execute(query, (key['key'],))

    results = []
    for row in cursor.fetchall():
        
        product_data = {
            "_key": row[0],
            "productName": row[1],
            "productDescription": row[2],
            "productPicture": row[3],
            "reviews": [],
            "variations": []
        }
        
        # Query to get list of category that product belongs to
        query = """
            SELECT c._key, c.categoryName, c.categoryPicture
            FROM product_categories pc
            INNER JOIN categories c ON pc.category_key = c._key
            WHERE pc.product_key = ?
        """
        cursor.execute(query, (product_data['_key'],))
        category_data = [
            {
                "_key": row[0],
                "categoryName": row[1],
                "categoryPicture": row[2],
            }
            for row in cursor.fetchall()
        ]

        # Query to get reviews (using product_data["_key"] directly)
        cursor.execute("SELECT userId, comment FROM Reviews WHERE productKey=?", (product_data["_key"],))
        for review in cursor.fetchall():
            product_data["reviews"].append({
                "userId": review[0],
                "comment": review[1]
            })

        # Query to get variations (using product_data["_key"] directly)
        cursor.execute("SELECT availabilityQuantity, discountPrice, offerPrice, quantity, sellingPrice FROM Variations WHERE productKey=?", (product_data["_key"],))
        for variation in cursor.fetchall():
            product_data["variations"].append({
                "availabilityQuantity": variation[0],
                "discountPrice": variation[1],
                "offerPrice": variation[2],
                "quantity": variation[3],
                "sellingPrice": variation[4]
            })

        results.append({
            "categoryDetails": category_data,
            "productDetails": product_data
        })

    # Close connection
    connection.close()

    return results

huy = get_product_from_key({'type' : 'category',
                      'key': 'best_seller'})
# huy.get('result')
for i in huy:
    print(i['productDetails']['reviews'])
    break

[{'userId': 'user_789', 'comment': 'Great phone!'}]


In [ ]:
[{
'categoryDetails': [
    {'_key': 'best_seller',
    'categoryName': 'Best Sellers',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'},
   {'_key': 'free_delivery',
    'categoryName': 'Free Delivery',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'}
   ],
'productDetails': {
      '_key': 'prod_456',
    'productName': 'Smartphone Z',
    'productDescription': 'A high-quality smartphone with advanced features.',
    'productPicture': 'http://127.0.0.1:5000/static/images/iphone11promax.png',
    'reviews': [
        {'userId': 'user_789', 'comment': 'Great phone!'}
    ],
    'variations': [
        {'availabilityQuantity': 10,
        'discountPrice': 0.0,
        'offerPrice': 999.0,
        'quantity': 1,
        'sellingPrice': 1099.0}
    ]
}
},
 {'categoryDetails': 
     [{'_key': 'best_seller',
    'categoryName': 'Best Sellers',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'},
   {'_key': 'free_delivery',
    'categoryName': 'Free Delivery',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'}
   ],
  'productDetails': {'_key': 'prod_001',
   'productName': 'Samsung Galaxy Z Flip',
   'productDescription': 'A flagship foldable smartphone from Samsung.',
   'productPicture': 'https://cdn0.vox-cdn.com/hermano/verge/product/image/9662/dseifert-4711-samsung-z-flip-3-noWM-3.jpg',
   'reviews': [{'userId': 'nyuhe.li', 'comment': 'Great phone!'}],
   'variations': [{'availabilityQuantity': 10,
     'discountPrice': 0.0,
     'offerPrice': 999.0,
     'quantity': 1,
     'sellingPrice': 1099.0}]}},
 {'categoryDetails': [{'_key': 'best_seller',
    'categoryName': 'Best Sellers',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'},
   {'_key': 'free_delivery',
    'categoryName': 'Free Delivery',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'}],
  'productDetails': {'_key': 'prod_002',
   'productName': 'Smartphone X',
   'productDescription': 'A revolutionary new smartphone with cutting-edge technology.',
   'productPicture': 'https://i0.wp.com/immrfabulous.com/wp-content/uploads/2020/03/img_9077-scaled.jpg?fit=1200,1162',
   'reviews': [{'userId': 'nyuhe.li', 'comment': 'Great phone!'}],
   'variations': [{'availabilityQuantity': 10,
     'discountPrice': 0.0,
     'offerPrice': 999.0,
     'quantity': 1,
     'sellingPrice': 1099.0}]}},
 {'categoryDetails': [{'_key': 'best_seller',
    'categoryName': 'Best Sellers',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'},
   {'_key': 'free_delivery',
    'categoryName': 'Free Delivery',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'}],
  'productDetails': {'_key': 'prod_003',
   'productName': 'Smartphone Y',
   'productDescription': 'An innovative smartphone with an immersive display.',
   'productPicture': 'https://cdn0.vox-cdn.com/hermano/verge/product/image/9662/dseifert-4711-samsung-z-flip-3-noWM-3.jpg',
   'reviews': [{'userId': 'nyuhe.li', 'comment': 'Great phone!'}],
   'variations': [{'availabilityQuantity': 10,
     'discountPrice': 0.0,
     'offerPrice': 999.0,
     'quantity': 1,
     'sellingPrice': 1099.0}]}},
 {'categoryDetails': [{'_key': 'best_seller',
    'categoryName': 'Best Sellers',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'},
   {'_key': 'free_delivery',
    'categoryName': 'Free Delivery',
    'categoryPicture': 'http://127.0.0.1:5000/static/images/tech.jpeg'}],
  'productDetails': {'_key': 'prod_004',
   'productName': 'Smartphone Z Pro',
   'productDescription': 'The ultimate smartphone experience with unparalleled performance.',
   'productPicture': 'https://cdn0.vox-cdn.com/hermano/verge/product/image/9662/dseifert-4711-samsung-z-flip-3-noWM-3.jpg',
   'reviews': [{'userId': 'nyuhe.li', 'comment': 'Great phone!'}],
   'variations': [{'availabilityQuantity': 10,
     'discountPrice': 0.0,
     'offerPrice': 999.0,
     'quantity': 1,
     'sellingPrice': 1099.0}]}}]

In [29]:
term = 'smart pHone        Z'
term = term.replace(" ", "").lower()
len(term[:int(len(term)/2)])
term[len(term)/2:]

'phonez'

In [2]:
'test_branch'

'test_branch'

In [12]:
test_hello_world()16:17:45,110 16:21:20,748

In [1]:
import requests
import json

# Base URL for your API
base_url = "https://nguyenduchuy.pythonanywhere.com" 
# base_url = 'http://192.168.0.101:8080'

url = base_url + '/users/check_user'
params ={"userId": "h@g.c",
         "password": "Huy12345!"}
print(url)
requests.post(url, json=params).text

https://nguyenduchuy.pythonanywhere.com/users/check_user


'{"result":false}\n'